In [1]:
import os
import sys
import cv2
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
import shutil

from ultralytics import YOLO
model = YOLO('../yolov8n-pose.pt')

import pandas as pd
import numpy as np

import json

In [2]:
def remove_file(folder_path = 'tmp'):
    [os.remove(f) for f in glob(folder_path)]

def get_train_data(origin_path, dst_path):
    shutil.copy(origin_path, dst_path)

In [3]:
remove_file(folder_path = '/kdh/talData/images/train/*')
remove_file(folder_path = '/kdh/talData/images/val/*')
remove_file(folder_path = '/kdh/talData/labels/train/*')
remove_file(folder_path = '/kdh/talData/labels/val/*')

In [4]:
# origin data paths
origin_image_path = glob('/kdh/talData/imageData/*.jpg')
origin_json_path = glob('/kdh/talData/labelData/*.json')

print(f'origin image length : {len(origin_image_path)}')
print(f'origin json length : {len(origin_json_path)}')
print('\n')


# setting for labels
zero_result, over_result = [], []
label_list = ['머리', '목', '오른쪽 어깨', '오른쪽 팔꿈치',
                '오른쪽 손목', '오른쪽 손끝', '왼쪽 어깨',
                '왼쪽 팔꿈치', '왼쪽 손목', '왼쪽 손끝',
                '허리(등)', '허리(요부)', '오른쪽 골반', '오른쪽 무릎',
                '오른쪽 발목', '오른쪽 발가락', '오른쪽 발끝', '왼쪽 골반',
                '왼쪽 무릎', '왼쪽 발목', '왼쪽 발가락', '왼쪽 발끝']

label_list = pd.DataFrame(label_list)
label_list.columns = ['keypointlabels']

# split train/val image/label
np.random.seed(seed = 4326)

val_image_paths = []
train_image_paths = np.random.choice(origin_image_path, 1000, replace = False)

for i in origin_image_path:
    if i not in train_image_paths:
        val_image_paths.append(i)

val_image_paths = np.array(val_image_paths)

print(f'train data length : {len(train_image_paths)}')
print(f'valid data length : {len(val_image_paths)}')

### prepare train data
for j in tqdm(train_image_paths):

    ## copy train image
    bucket = '/kdh/talData/'

    image_train_folder = 'images/train/'
    label_train_folder = 'labels/train/'

    file_name = j.split('/')[-1]

    ## make train label data
    with open(bucket + 'labelData/' + file_name.split('.')[0] + '.json', 'r') as file:
        data = json.load(file)

    tmp = pd.DataFrame(data['annotation_info']['keypoint'])
    boxes_predict = model(j)

    if (len(tmp) != 23) & (len(tmp) != 0):
        shutil.copy(j, bucket + image_train_folder + file_name)

        tmp['x'] = tmp['x'] / 3840
        tmp['y'] = tmp['y'] / 2160

        tmp = pd.merge(label_list, tmp, on = 'keypointlabels', how = 'left')
        tmp = tmp.fillna(-1)

        x_tmp = np.array(tmp['x'])
        y_tmp = np.array(tmp['y'])

        with open(bucket + label_train_folder + file_name.split('.')[0] + '.txt', 'w') as file:
            file.write('0 ')
            for m in range(0, 4, 1):
                try:
                    file.write(str(boxes_predict[0].boxes.xywhn.cpu()[0][m].item()) + ' ')
                except:
                    file.write('-1 -1 -1 -1 ')

            for l in range(0, 22 ,1):
                file.write(str(x_tmp[l]) + ' ')
                file.write(str(y_tmp[l]) + ' ')

    elif len(tmp) == 0:
        zero_result.append(j)

    elif len(tmp) == 23:
        over_result.append(j)



### prepare valid data
for k in tqdm(val_image_paths):
    bucket = '/kdh/talData/'

    image_val_folder = 'images/val/'
    label_val_folder = 'labels/val/'

    file_name = k.split('/')[-1]


    ## make train label data
    with open(bucket + 'labelData/' + file_name.split('.')[0] + '.json', 'r') as file:
        data = json.load(file)

    tmp = pd.DataFrame(data['annotation_info']['keypoint'])
    boxes_predict = model(k)

    if (len(tmp) != 23) & (len(tmp) != 0):
        shutil.copy(k, bucket + image_val_folder + file_name)

        tmp['x'] = tmp['x'] / 3840
        tmp['y'] = tmp['y'] / 2160

        tmp = pd.merge(label_list, tmp, on = 'keypointlabels', how = 'left')
        tmp = tmp.fillna(-1)

        x_tmp = np.array(tmp['x'])
        y_tmp = np.array(tmp['y'])

        with open(bucket + label_val_folder + file_name.split('.')[0] + '.txt', 'w') as file:
            file.write('0 ')
            for m in range(0, 4, 1):
                try:
                    file.write(str(boxes_predict[0].boxes.xywhn.cpu()[0][m].item()) + ' ')
                except:
                    file.write('-1 -1 -1 -1 ')
                    
            for l in range(0, 22 ,1):
                file.write(str(x_tmp[l]) + ' ')
                file.write(str(y_tmp[l]) + ' ')

    elif len(tmp) == 0:
        zero_result.append(l)

    elif len(tmp) == 23:
        over_result.append(l)

origin image length : 1224
origin json length : 1224


train data length : 1000
valid data length : 224


  0%|          | 0/1000 [00:00<?, ?it/s]
/home/dongha/miniconda3/envs/go/lib/python3.8/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
image 1/1 /kdh/talData/imageData/A05_B078_LC_01_05_frame18045.jpg: 384x640 1 person, 96.8ms
Speed: 4.7ms preprocess, 96.8ms inference, 86.2ms postprocess per image at shape (1, 3, 384, 640)
  0%|          | 1/1000 [00:01<18:24,  1.11s/it]
image 1/1 /kdh/talData/imageData/A05_B069_LC_01_05_frame12105.jpg: 384x640 1 person, 5.8ms
Speed: 2.0ms preprocess, 5.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
  0%|          | 2/1000 [00:01<08:50,  1.88it/s]
image 1/1 /kdh/talData/imageData/A05_B078_LC_01_05_frame17775.jpg: 384x640 1 person, 5.3ms
Speed: 1.9ms preprocess, 5.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
  0%|          | 

In [21]:
key_tmp[0].loc[:, ['x', 'y', 'keypointlabels']]

,x,y,keypointlabels
0,0.258833,0.193825,머리
1,0.289052,0.200021,목
2,0.266969,0.212412,오른쪽 어깨
3,0.229196,0.252685,오른쪽 팔꿈치
4,0.176894,0.279533,오른쪽 손목
5,0.168759,0.333230,오른쪽 손끝
6,0.318108,0.190727,왼쪽 어깨
7,0.329730,0.143226,왼쪽 팔꿈치
8,0.315202,0.111214,왼쪽 손목
9,0.309391,0.137030,왼쪽 손끝


In [22]:
tmp.loc[:, ['x', 'y', 'keypointlabels']]

,x,y,keypointlabels
0,0.413085,0.387150,머리
1,-1.000000,-1.000000,목
2,0.390729,0.370100,오른쪽 어깨
3,0.371566,0.420436,오른쪽 팔꿈치
4,0.390729,0.438298,오른쪽 손목
5,0.408066,0.457783,오른쪽 손끝
6,0.450497,0.347368,왼쪽 어깨
7,0.462360,0.422060,왼쪽 팔꿈치
8,0.424947,0.447228,왼쪽 손목
9,0.403960,0.471584,왼쪽 손끝


In [23]:
key_tmp[7].loc[:, ['x', 'y', 'keypointlabels']]

,x,y,keypointlabels
0,0.414480,0.203116,머리
1,0.428627,0.264896,오른쪽 팔꿈치
2,0.421553,0.268328,오른쪽 손목
3,0.444059,0.198540,왼쪽 손끝
4,0.460134,0.293497,허리(등)
5,0.464635,0.371293,허리(요부)
6,0.444702,0.494852,오른쪽 무릎
7,0.458205,0.576080,오른쪽 발목
8,0.450489,0.580656,오른쪽 발가락
9,0.443416,0.578368,오른쪽 발끝


In [8]:
print(f'non-keypoints number : {len(zero_result)}')
print(f'23-keypoints number : {len(over_result)}')

non-keypoints number : 25
23-keypoints number : 2


In [19]:
print(len(glob('/kdh/talData/images/train/*.jpg')))
print(len(glob('/kdh/talData/images/val/*.jpg')))
print(len(glob('/kdh/talData/labels/train/*.txt')))
print(len(glob('/kdh/talData/labels/val/*.txt')))

976
221
976
221


In [70]:
remove_file(folder_path = '/home/kdh/Desktop/project/test_data/images/train/*.jpg')
remove_file(folder_path = '/home/kdh/Desktop/project/test_data/images/val/*.jpg')
remove_file(folder_path = '/home/kdh/Desktop/project/test_data/labels/train/*.txt')
remove_file(folder_path = '/home/kdh/Desktop/project/test_data/labels/val/*.txt')

default_path = '/home/kdh/Desktop/project/'
TARGET = 1000

for i in range(0, TARGET, 1):
    file_name = np_origin_key_data[i][0].split('.')[0] + '.txt'
    tmp = model(default_path + 'tal_data/camera1/color/color/' + np_origin_key_data[i][0])

    shutil.copy(default_path + 'tal_data/camera1/color/color/' + np_origin_key_data[i][0],
                default_path + 'test_data/images/train/' + np_origin_key_data[i][0])

    with open(default_path + f'test_data/labels/train/{file_name}', 'w') as file:
        file.write('0 ')
        for j in range(0, 4, 1):
            file.write(str(tmp[0].boxes.xywhn.cpu()[0][j].item()) + ' ')

        for k in range(1, len(np_origin_key_data[i]), 1):
            file.write(str(np_origin_key_data[i][k]))
            file.write(' ')


for i in range(TARGET, TARGET + 100, 1): 
    file_name = np_origin_key_data[i][0].split('.')[0] + '.txt'
    tmp = model(default_path + 'tal_data/camera1/color/color/' + np_origin_key_data[i][0])

    shutil.copy(default_path + 'tal_data/camera1/color/color/' + np_origin_key_data[i][0],
                default_path + 'test_data/images/val/' + np_origin_key_data[i][0])

    with open(default_path + f'test_data/labels/val/{file_name}', 'w') as file:
        file.write('0 ')
        for j in range(0, 4, 1):
            file.write(str(tmp[0].boxes.xywhn.cpu()[0][j].item()) + ' ')

        for k in range(1, len(np_origin_key_data[i]), 1):
            file.write(str(np_origin_key_data[i][k]))
            file.write(' ')


image 1/1 /home/kdh/Desktop/project/tal_data/camera1/color/color/0_1637116298835.jpg: 384x640 7 persons, 5.2ms
Speed: 15.1ms preprocess, 5.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/kdh/Desktop/project/tal_data/camera1/color/color/1_1637116298874.jpg: 384x640 9 persons, 5.3ms
Speed: 1.3ms preprocess, 5.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/kdh/Desktop/project/tal_data/camera1/color/color/2_1637116298902.jpg: 384x640 7 persons, 5.2ms
Speed: 1.3ms preprocess, 5.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/kdh/Desktop/project/tal_data/camera1/color/color/3_1637116298940.jpg: 384x640 11 persons, 6.2ms
Speed: 1.3ms preprocess, 6.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/kdh/Desktop/project/tal_data/camera1/color/color/4_1637116298973.jpg: 384x640 7 persons, 5.4ms
Speed: 1.5ms preprocess, 5.4ms inference, 1.3ms postpr

In [4]:
model = YOLO('../yolov8n-pose.pt')

results = model.train(data = 'talDance-pose.yaml', epochs = 100, imgsz = 850)

New https://pypi.org/project/ultralytics/8.0.162 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.147 🚀 Python-3.8.17 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 9987MiB)
engine/trainer: task=pose, mode=train, model=../yolov8n-pose.pt, data=talDance-pose.yaml, epochs=100, patience=50, batch=16, imgsz=850, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, clas